In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from livelossplot import PlotLosses
from tqdm import tqdm
from models.foster import VariationalAutoencoder

In [ ]:
BATCH_SIZE = 512
LEAKY_RELU_NEGATIVE_SLOPE = 0.1
DROPOUT_P = 0.25
Z_DIM = 2
LEARNING_RATE = 0.005
NUM_EPOCHS = 200

RECON_LOSS_FACTOR = 500

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_dataset = MNIST(root='data/', train=True, transform=transforms.ToTensor(), download=True)
validation_dataset = MNIST(root='data/', train=False, transform=transforms.ToTensor(), download=True)

In [ ]:
dataloaders = {'train': DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True),
               'validation': DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)}

In [ ]:
model = VariationalAutoencoder(z_dim=Z_DIM, leaky_relu_negative_slope=LEAKY_RELU_NEGATIVE_SLOPE, dropout_p=DROPOUT_P)

#initialize with AE parameters
model.encoder.mu_encoder = torch.load('checkpoints/ae_encoder_only.pt')
model.decoder = torch.load('checkpoints/ae_decoder_only.pt')

model = model.to(device)
model(torch.rand(BATCH_SIZE, 1, 28, 28).to(device))  # test forward pass
model.summary(input_data=torch.rand((BATCH_SIZE, 1, 28, 28)))  # summarize

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [ ]:
liveloss = PlotLosses()
logs = {}
for epoch in tqdm(range(NUM_EPOCHS)):
    for phase in ['train', 'validation']:
        model.train() if phase == 'train' else model.eval()
        running_recon_loss = 0.0
        running_kl_loss = 0.0
        running_loss = 0.0
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            outputs, recon_loss, kl_loss = model(inputs)
            recon_loss = recon_loss
            loss = RECON_LOSS_FACTOR * recon_loss + kl_loss
            if phase == 'train':
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            running_recon_loss += recon_loss.detach() * inputs.size(0)
            running_kl_loss += kl_loss.detach() * inputs.size(0)
            running_loss += loss.detach() * inputs.size(0)
        epoch_recon_loss = running_recon_loss / len(dataloaders[phase].dataset)
        epoch_vq_loss = running_kl_loss / len(dataloaders[phase].dataset)
        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        prefix = 'val_' if phase == 'validation' else ''
        logs[prefix + 'recon_loss'] = epoch_recon_loss.item()
        logs[prefix + 'kl_loss'] = epoch_vq_loss.item()
        logs[prefix + 'loss'] = epoch_loss.item()
    if epoch > 3:
        liveloss.update(logs)
        liveloss.send()
torch.save(model, 'checkpoints/vae.pt')
torch.save(model.encoder.mu_encoder, 'checkpoints/vae_mu_encoder_only.pt')
torch.save(model.decoder, 'checkpoints/vae_decoder_only.pt')